# Exploring the Coronavirus (SARS-COV-2) genome

We will whole genome sequence of the original isolate of the SARS-COV-2 virus from [GenBank](https://www.ncbi.nlm.nih.gov/genbank/). While this is a RNA virus, we indicate "dna" just for convenience. (This just swaps the RNA character "U" for the DNA character "T" in the sequence.)

The data format downloaded from GenBank includes bothe sequence and genomne "features"

In [ ]:
import pathlib
import cogent3


def download_genbank_seq(accession: str, outpath: pathlib.Path):
    url = f"http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id={accession}&rettype=gb&retmode=text"
    with cogent3.open_(url, mode="rb") as infile:
        data = infile.read()

    with cogent3.open_(outpath, mode="wb") as outfile:
        outfile.write(data)


alpha_path = pathlib.Path("alpha.gb.gz")
if not alpha_path.exists():
    download_genbank_seq("NC_045512.2", alpha_path)

alpha = cogent3.load_seq(alpha_path, moltype="dna")

We display the genetic organisation of the SARS-COV-2 genome. We use Plotly for the visualisation. If you hover your cursor over an element in the plot, the "symbol" is displayed. Note the the "S" gene (which encodes the Spike protein) is the second from the left. (The arrows indicate the direction in which the gene is encoded.)

In [ ]:
fig = alpha.get_drawable(
    biotype=("gene", "5'UTR", "3'UTR"),
)
fig.layout.title = "SARS-CoV-2 genome"
fig.layout |= dict(width=1000, height=300)
fig.write("sars_cov2_genome.svg")
fig.write("sars_cov2_genome.png")
fig.show()

We use the `cogent3` sequence to get the feature for the "S" gene.

In [ ]:
feature = list(alpha.get_features(name="S", biotype="gene"))[0]
feature

We can use that feature to get the nucleic acid sequence

In [ ]:
alpha_spike = alpha[feature]
alpha_spike

which can then be translated it into protein sequence.

In [ ]:
alpha_spike.get_translation()

Much of genomics relies on looking for evidence of natural selection in genetic variation to tell us where the important information is. At its simplest level, this involves comparing the genomes of two organisms. To illustrate this, I will download the genome for a different SARS-COV-2 strain (Omicron) and extract it's Spike gene. I put the two into a single sequence collection.

In [ ]:
omicron_path = pathlib.Path("omicron.gb.gz")
if not omicron_path.exists():
    download_genbank_seq("OR575624.1", omicron_path)

omicron = cogent3.load_seq(omicron_path, moltype="dna")
feature = list(omicron.get_features(name="S", biotype="gene"))[0]
omicron_spike = omicron[feature]
coll = cogent3.make_unaligned_seqs({"alpha": alpha, "omicron": omicron}, moltype="dna")
coll

I use a "Dotplot" to visualise the relationship between the genomes at the individual character level. If there were no genetic differences, we would expect a single blue line on the diagnoal. Where that line is broken indicates the location of a difference in their genome sequence.

In [ ]:
spike_seqs = cogent3.make_unaligned_seqs(
    {"alpha": alpha_spike, "omicron": omicron_spike}, moltype="dna"
)

dp = spike_seqs.dotplot(window=20, threshold=20, k=10, title="Spike gene")
dp.remove_track(left_track=True)
dp.show(width=1000, height=1000)
dp.write("spike_gene_dotplot.svg")
dp.write("spike_gene_dotplot.png")

The corresponding plotly figure can of a `cogent3` drawables can be accessed directly via a `.plotly_figure` attribute.

In [ ]:
print(type(dp.plotly_figure))
dp.plotly_figure

To do a more comprehensive analysis of genetic variation requires that we have a larger collection of related sequences. Of particular importance is that these sequences have been subjected to a multiple sequence alignment algorithm. The upshot of this algorithm is we have a matrix of sequences. Rows in this matrix correspond to genomes. The columns correspond to the characters that have decended from the same character in the common ancestor of all the genomes in the collection. We download a publicly available multiple sequence alignment. We want to analyze just the spike gene segment of this alignment. We project the coordinates from one of the genomes annotations present into the alignment coordinate system. We use those coordinates to slice the alignment.

We also want to eliminate sequences that have a lot of missing data. `cogent3` represents the sequences as a numpy array, for DNA (and RNA) "missing" or incomplete data will have indices > 4. So we exclude genomes whose Spike gene has > 1% missing data.

In [ ]:
import pathlib

import cogent3


def get_sars_alignment(outpath):
    if outpath.exists():
        return cogent3.load_aligned_seqs(outpath, moltype="dna")

    url = "https://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/UShER_SARS-CoV-2/public-latest.all.msa.fa.xz"
    aln = cogent3.load_aligned_seqs(
        url,
        format="fasta",
        label_to_name=lambda x: x.split("|")[-2],
        moltype="dna",
    )
    aln.write(outpath, file_format="fasta", block_size=100_000)
    return aln


def get_spike_coords_from_known(aln):
    seqid = "PP692424.1"
    selected = aln.take_seqs([seqid])
    # based on PP692424.1 GenBank record, plus discovering error in annotation
    selected.annotation_db.add_feature(
        seqid=seqid, biotype="gene", name="Spike", spans=[(21469, 25264)]
    )
    f = list(selected.get_features(seqid=seqid, name="Spike"))[0]
    # these are the start and end of the gene in alignment coordinates
    return f.map.start, f.map.end


def sars():
    import numpy

    aln_path = pathlib.Path("public-latest.all.msa.fa")
    aln = get_sars_alignment(aln_path)
    # s, e = get_spike_coords_from_known(aln)
    # hard coding the result of that call below
    s, e = 21469, 25264
    spike = aln[s:e]
    # select sequences that have < 1% missing data in S gene
    # for DNA / RNA, "missing data" characters have indices > 4
    spike = spike.take_seqs_if(
        lambda seq: (numpy.array(seq) > 4).sum() / len(spike) < 0.01
    )
    # removing redundant gaps
    spike = spike.omit_gap_pos(allowed_gap_frac=1 / spike.num_seqs)
    return spike

In the interests of cutting to the chase, we limit the analysis to the following "slice" (which is a subset of aligned positions).

In [ ]:
# this step takes a few seconds as it's cleaning up data
# reducing the alignment down to a smaller region after discovering this
# patch of coevolving sites
aln = sars()[1925:2050]

We then perform the coevolution analysis. This produces a pairwise matrix, an object which has a `drawable` attribute. In this case, this wraps production of a Plotly heatmap. We use the RMI (resamling Mutual Information statistic, described in [this paper](http://www.ncbi.nlm.nih.gov/pubmed/19055758)) to measure coevolution, the values are between 0 and 1. The higher the value, the more likely the two positions are to be coevolving.

Note the two clusters of high coevolution values are also co-evolving with each other.

In [ ]:
dmat = aln.coevolution(stat="rmi", drawable="heatmap", show_progress=False)

In [ ]:
fig = dmat.drawable
fig.layout.title = "Clustered coevolving positions within SPIKE"
fig.show()

To save these images to a static file, use the `write()` method on the `cogent3` drawable.